# Selenium - 동적 웹페이지 스크래핑

In [ ]:
# Seleium 드라이버 생성
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 화면없이 실행

# 드라이버 서비스 생성
service = Service(ChromeDriverManager().install())

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

## 뉴스 서비스에 접속

In [ ]:
# 뉴스 사이트
url = "https://news.daum.net/"

driver.get(url)

In [ ]:
# 페이지 소스 가져오기
from bs4 import BeautifulSoup
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

# 태그 검색
print('title 태그 요소: ', soup.title)
print('title 태그 이름: ', soup.title.name)
print('title 태그 문자열: ', soup.title.text)

## 경제 카테고리를 선택

In [ ]:
from selenium.webdriver.common.by import By

# 경제 뉴스 클릭
economy = driver.find_element(By.CSS_SELECTOR, '#gnbContent > div > ul > li:nth-child(4) > a > span')
economy

In [ ]:
economy.click()

driver.implicitly_wait(5)

In [ ]:
# 페이지 소스 가져오기

soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

In [ ]:
news[0]

In [ ]:
news[0].select("a.link_txt")[0].text.strip()

In [ ]:
news[0].select_one("a.link_txt").text.strip()

In [ ]:
# 데이터 추출하여 정리
data = {'title':[], 'agency':[], 'category':[], 'link':[]}

for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
import pandas as pd
df = pd.DataFrame(data)
df.head()

In [ ]:
len(df)

## 페이지 이동하면서 반복적으로 수집

In [ ]:
# 페이지네이션 배열
#timeline > div > div > a

page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
len(page_numbers)

In [ ]:
page_numbers[0]

In [ ]:
for p in page_numbers:
    print(p.text)

In [ ]:
current_page = 1

In [ ]:
for p in page_numbers:
    print(p.text)
    if (p.text != '다음') and (int(p.text) == current_page + 1):
        p.click()
        driver.implicitly_wait(5)
        current_page = current_page + 1
        print(p.text, current_page)
        break

In [ ]:
current_page

In [ ]:
# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

In [ ]:
# 뉴스 추출하여 기존 데이터에 추가
for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
df = pd.DataFrame(data)
print(len(df))

df.head()

In [ ]:
# 드라이버 종료
driver.close()

In [ ]:
# 페이지를 이동하면서 데이터를 수집하여 정리하는 함수
def get_news(driver, category):

    data = {'title':[], 'agency':[], 'category':[], 'link':[]}

    current_page = 1
    
    while current_page < 6:
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            news = soup.select('#timeline > ul > li')
            
            for item in news:
                title = item.select_one('strong > a').text
                agency = item.select_one('span.info_cp').text
                link = item.select_one('strong > a')['href']
                
                data['title'].append(title)
                data['agency'].append(agency)
                data['category'].append(category)
                data['link'].append(link)
            
            page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
            driver.implicitly_wait(5)
            
            for p in page_numbers:
                if (p.text != '다음') and (int(p.text) == current_page + 1):
                    p.click()
                    driver.implicitly_wait(5)
                    current_page = current_page + 1
                    print(p.text, current_page)
                    break        

        except:
            pass
    
    df = pd.DataFrame(data)
    
    driver.close()

    return df

In [ ]:
# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# 뉴스 사이트 접속
url = "https://news.daum.net/"
driver.get(url)

In [ ]:
# 뉴스 카테고리 메뉴 배열
category_tabs = driver.find_elements(By.CSS_SELECTOR, '#gnbContent > div > ul > li')
category_tabs

In [ ]:
category_tabs[3].text

In [ ]:
category_dict = {item.text:item for item in category_tabs}
category_dict

In [ ]:
# 경제 뉴스 클릭
category_dict['경제'].click()
driver.implicitly_wait(5)

In [ ]:
# 데이터 수집
df_economy = get_news(driver, '경제')
print(len(df_economy))

df_economy.head()

In [ ]:
# 페이지 이동할 때 동적으로 기사 배열이 바뀌면서 일부 뉴스가 중복 수집되는 현상이 있습니다. 중복을 제거합니다. 
df_economy = df_economy.drop_duplicates()
len(df_economy)

In [ ]:
# 데이터 저장
df_economy.to_csv('daum_economy_news.csv', index=False)